In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np


DEVICE = torch.device('cpu')
OUTPUT_SIZE = 2048

model = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.IMAGENET1K_V2)

extraction_layer = model._modules.get('avgpool')
model.to(DEVICE)
model.eval()

scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_vec(arg, model, extraction_layer):
    image = normalize(to_tensor(scaler(arg))).unsqueeze(0).to(DEVICE)
    result = torch.zeros(1, OUTPUT_SIZE, 1, 1)
    def copy_data(m, i, o):
        result.copy_(o.data)
    hooked = extraction_layer.register_forward_hook(copy_data)
    with torch.no_grad():
        model(image)
    hooked.remove()
    return result

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-1a0047aa.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-1a0047aa.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 125MB/s]


In [2]:
import arrow
import base64
import pandas as pd
from glob import iglob
from io import BytesIO
from os.path import basename
from os.path import dirname
from PIL import Image

ROOT = '/kaggle/input/mtarsidataset/MTARSI_Dataset'
TEST_IMAGES = ROOT + '/test'
THUMBNAIL_SIZE = (128, 128)
TRAIN_IMAGES = ROOT + '/train'


def embed(model, filename: str):
    with Image.open(fp=filename, mode='r') as image:
        return get_vec(arg=image.convert('RGB'), model=model, extraction_layer=extraction_layer).numpy().reshape(OUTPUT_SIZE,)

def png(filename: str) -> str:
    with Image.open(fp=filename, mode='r') as image:
        buffer = BytesIO()
        # our images are pretty big; let's shrink the hover images to thumbnail size
        image.resize(size=THUMBNAIL_SIZE).convert('RGB').save(buffer, format='png')
        return 'data:image/png;base64,' + base64.b64encode(buffer.getvalue()).decode()

time_start = arrow.now()
train_df = pd.DataFrame(data=[pd.Series(data=[basename(item), embed(model=model, filename=item), png(filename=item), basename(dirname(item))],
                                        index=['item', 'value', 'png', 'label']) for item in iglob(TRAIN_IMAGES + '/*/*.jpeg')])
print('done loading training data in {}'.format(arrow.now() - time_start))
test_df = pd.DataFrame(data=[pd.Series(data=[basename(item), embed(model=model, filename=item), png(filename=item), basename(dirname(item))],
                                        index=['item', 'value', 'png', 'label']) for item in iglob(TEST_IMAGES + '/*/*.jpeg')])
print('done in {}'.format(arrow.now() - time_start))

done loading training data in 0:20:01.792344
done in 0:25:01.979784
